In [1]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms.huggingface_pipeline import HuggingFacePipeline

c:\Conda\envs\langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name: str = "microsoft/phi-2"  

nf4_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = nf4_config,
    low_cpu_mem_usage = True
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:21<00:00, 10.61s/it]


In [5]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
max_new_token = 1024

model_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=max_new_token,
    pad_token_id = tokenizer.eos_token_id
)

In [6]:
gen_kwargs = {
    "temperature": 0.6
}

llm = HuggingFacePipeline(pipeline = model_pipeline, model_kwargs = gen_kwargs) 

C:\Users\FPT Shop\AppData\Local\Temp\ipykernel_16800\680517590.py:5: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFacePipeline`.
  llm = HuggingFacePipeline(pipeline = model_pipeline, model_kwargs = gen_kwargs)


In [9]:
llm.invoke("what day is today")

'what day is today")\n    print(f"{today} is a {day_of_week}")\n    print(f"{today} is a {day_of_week_ordinal}")\n    print(f"{today} is a {day_of_week_ordinal_ordinal}")\n    print(f"{today} is a {day_of_week_ordinal_ordinal_ordinal}")\n    print(f"{today} is a {day_of_week_ordinal_ordinal_ordinal_ordinal}")\n    print(f"{today} is a {day_of_week_ordinal_ordinal_ordinal_ordinal_ordinal}")\n    print(f"{today} is a {day_of_week_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal}")\n    print(f"{today} is a {day_of_week_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal}")\n    print(f"{today} is a {day_of_week_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal}")\n    print(f"{today} is a {day_of_week_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal}")\n    print(f"{today} is a {day_of_week_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal_ordinal}")\n    print(f"{today} is a {day_of_week_ordinal_ordinal_ordinal_ordinal_ordina

1.PromptTemplate


In [11]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """Instruct: {prompt}\nOutput:"""
)

user_prompt = "Write a detailed analogy between mathematics and a lighthouse."
messages = prompt_template.format(prompt=user_prompt)

In [13]:
output = llm.invoke(messages)
print(output)

Instruct: Write a detailed analogy between mathematics and a lighthouse.
Output: Mathematics is to logic as a lighthouse is to navigation.



2.Chatprompt template


In [14]:
from langchain_core.prompts import  ChatPromptTemplate

In [23]:
messages = [
    ("system", "You are a helpful AI bot.Your name is {bot_name}."),
    ("user", "Hi!"),
    ("assistant", "Hello! How can I help you today?"),
    ("user", "{user_input}"),
]

In [24]:
chat_template = ChatPromptTemplate.from_messages(messages)

In [27]:
messages = chat_template.format_messages(
    bot_name = "Halo",
    user_input = "What is the weather like today?"
)

In [29]:
print(type(messages))

<class 'list'>


In [30]:
output = llm.invoke(messages)
print(output)

System: You are a helpful AI bot.Your name is Halo.
Human: Hi!
AI: Hello! How can I help you today?
Human: What is the weather like today?
AI: The weather today is sunny with a high of 75 degrees Fahrenheit. Would you like to know more about the weather forecast?



chain

In [41]:
prompt = PromptTemplate.from_template(
    """Instruct: {prompt}\nOutput:"""   
)

# Define a chain
chain = prompt | llm

output = chain.invoke(
    {
        "prompt": "Hi",
    }
)

print(output)

Instruct: Hi
Output: Hi there, how are you doing today?

